In [ ]:
!pip install langchain -q
!pip install transformers huggingface-hub -q
!pip install -q bitsandbytes accelerate
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.4 MB/s eta 0:00:00


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model_id = "daryl149/llama-2-7b-chat-hf"

model_load = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [44]:
from transformers import pipeline

pipeline = pipeline(
    "text-generation", #task
    model=model_load,
    tokenizer=tokenizer,
    top_k=10,
    max_length = 4000,
    do_sample=True,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

In [45]:
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline(
    pipeline = pipeline,
    model_kwargs={"temperature" : 0})

In [5]:
!git clone https://github.com/TheMITTech/shakespeare

Cloning into 'shakespeare'...
remote: Enumerating objects: 2187, done.
remote: Total 2187 (delta 0), reused 0 (delta 0), pack-reused 2187
Receiving objects: 100% (2187/2187), 5.95 MiB | 20.78 MiB/s, done.
Resolving deltas: 100% (1135/1135), done.


Let's collect all the HTML documents into a single directory.

In [6]:
from glob import glob

files = glob("./shakespeare/**/*.html")

In [7]:
import shutil
import os

os.mkdir('./data')
destination_folder = './data/'

for html_file in files:
  shutil.move(html_file, destination_folder + html_file.split("/")[-1])

In [8]:
# install dependencies
!pip install beautifulsoup4 -q

In [9]:
from langchain.document_loaders import BSHTMLLoader, DirectoryLoader

bshtml_dir_loader = DirectoryLoader('./data/', loader_cls=BSHTMLLoader)

data = bshtml_dir_loader.load()

In [39]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(tokenizer, chunk_size=1000, chunk_overlap=20, separator="\n")

documents = text_splitter.split_documents(data)

Now that we have our documents split - we need to convert them into embeddings!

True to staying "on-prem" we will be using Hugging Face Hub embedding models to achieve this goal!

Remember, everything we're using today is only an example - feel free to extend/modify wherever you see fit!

In [29]:
# we'll need the sentence transformers library to utilize HF's Sentence XFormer for our embeddings.
!pip install sentence_transformers -q

NotImplementedError: ignored

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

In [13]:
# we'll need a few dependencies before we can do this
!pip install chromadb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━

In [30]:
from langchain.vectorstores import Chroma

persist_directory = "vector_db"

vectordb = Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory=persist_directory)

In [31]:
doc_retriever = vectordb.as_retriever()

### Final Chain

With that set-up, we're good to set-up our final chain and leverage all the documents we have in our Vector DB!

In [46]:
from langchain.chains import RetrievalQA

shakespeare_qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=doc_retriever)

Let's test it out by itself!

In [20]:
shakespeare_qa.run("Could you provide information about the main characters of the novel 'Hamlet'?")

" 'Hamlet' is a tragedy written by William Shakespeare, and its main characters are:\n1. Prince Hamlet: The protagonist and narrator of the play. A university student in Wittenberg, he returns home to Denmark after his father's death and finds that his mother has married his uncle. He is a melancholy and indecisive young man who is slow to act upon his revenge plan.\n2. King Claudius: Hamlet's uncle and stepfather, who marries Hamlet's mother, Queen Gertrude, after King Hamlet's death. He is a ruthless and power-hungry king who is responsible for King Hamlet's death and becomes the target of Hamlet's revenge.\n3. Queen Gertrude: Hamlet's mother, who marries King Claudius after King Hamlet's death. She is a complex character who is both vulnerable and guilty of her actions.\n4. Polonius: The Lord Chamberlain and Chief Advisor to King Claudius. He is a wise and well-spoken man who is also the father of Laertes and Ophelia. He is killed by Hamlet in a mistaken belief that he is the king.\

In [18]:
shakespeare_qa.run("Could you provide information about the main characters of the novel 'Hamlet'?")

'  Sure! The main characters of \'Hamlet\' are:\n\n\n\n\nHamlet, Prince of Denmark: the protagonist and titular character of the play. He is a young prince who returns to Denmark to find his father\'s ghost and avenge his murder. He is initially indecisive and melancholic, but eventually becomes determined to kill his uncle, Claudius, who he believes murdered his father.\n\n\n\n\nClaudius, King of Denmark: Hamlet\'s uncle and the murderer of Hamlet\'s father. He is a cunning and manipulative ruler who is afraid that Hamlet will reveal his crime.\n\n\n\n\n\nOphelia: Hamlet\'s love interest and the daughter of Polonius. She is a gentle and innocent young woman who is unsuited to the treachery and deceit of the royal court.\n\n\n\n\nPolonius: The Lord Chancellor of Denmark and Ophelia\'s father. He is a wise and well-meaning man who is often at odds with Hamlet\'s actions.\n\n\n\n\n\nGertrude, Queen of Denmark: Hamlet\'s mother and the wife of Claudius. She is a complex character who is t

In [47]:
shakespeare_qa.run("Could you summarize the content of the novel 'Hamlet' for me?")

OutOfMemoryError: ignored

### Conclusion

Here we have it!

A system capable of querying over multiple documents - all without every needing to hit an external API!